In [23]:
import torch
import yaml
import argparse
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from model import RNN
import pandas  as pd
from providers import MolecularNotationDataset, SpecialTokenWrapperModel, SmilesModel, robust_standardizer
from legogram import LegoGram
from legogram.apps import LegoGramRNNSampler
from torch.nn.utils.rnn import pad_sequence
import os
from torch.utils.tensorboard import SummaryWriter
import json

In [2]:
dataset = torch.load("lg.bin")

In [37]:
def collect(sequences):
    mols = [elem[0] for elem in sequences]
    endpoints = [elem[1] for elem in sequences]
    mols = sorted(mols,key=len,reverse=True)
    lengths = [seq.size(0) for seq in mols]
    return pad_sequence(mols,batch_first=True), torch.stack(endpoints), lengths


In [38]:
train_loader = DataLoader(dataset, batch_size=128, collate_fn=collect)

In [39]:
l = list(train_loader)

In [40]:
a = l[0]

In [41]:
a[0]

tensor([[ 1, 36, 41,  ...,  3,  4,  2],
        [ 1, 36, 41,  ...,  4,  2,  0],
        [ 1, 10, 44,  ...,  3,  2,  0],
        ...,
        [ 1, 36, 41,  ...,  0,  0,  0],
        [ 1, 10, 16,  ...,  0,  0,  0],
        [ 1, 10,  8,  ...,  0,  0,  0]])

In [42]:
a[1]

tensor([[ 30, 277,  -2],
        [ 60, 280,  -3],
        [130, 345,  -4],
        [140, 359,  -4],
        [170, 340,  -2],
        [140, 319,  -2],
        [160, 357,  -4],
        [180, 334,  -2],
        [ 70, 269,  -2],
        [190, 380,  -4],
        [ 60, 274,  -3],
        [160, 335,  -3],
        [120, 281,  -2],
        [ 40, 332,  -5],
        [130, 350,  -3],
        [ 70, 319,  -4],
        [170, 342,  -3],
        [ 60, 275,  -2],
        [140, 356,  -3],
        [120, 363,  -4],
        [110, 336,  -3],
        [ 70, 262,  -3],
        [ 70, 277,  -2],
        [140, 323,  -2],
        [110, 304,  -3],
        [120, 356,  -3],
        [140, 305,  -3],
        [120, 301,  -2],
        [ 90, 327,  -3],
        [120, 308,  -3],
        [ 90, 328,  -3],
        [ 90, 290,  -3],
        [110, 285,  -3],
        [120, 351,  -4],
        [ 30, 273,  -3],
        [100, 291,  -4],
        [100, 340,  -4],
        [110, 322,  -3],
        [120, 292,  -4],
        [100, 349,  -4],


In [43]:
a = 

[39,
 38,
 38,
 37,
 35,
 34,
 34,
 34,
 34,
 34,
 34,
 33,
 33,
 33,
 33,
 33,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 30,
 30,
 30,
 30,
 30,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 26,
 26,
 26,
 26,
 26,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 23,
 23,
 23,
 23,
 23,
 22,
 22,
 22,
 21,
 21,
 21,
 20,
 20,
 20,
 20,
 20,
 20,
 19,
 19,
 19,
 18,
 18,
 17,
 16,
 12]

In [36]:
torch.stack(a[1])

tensor([[ 30, 277,  -2],
        [ 60, 280,  -3],
        [130, 345,  -4],
        [140, 359,  -4],
        [170, 340,  -2],
        [140, 319,  -2],
        [160, 357,  -4],
        [180, 334,  -2],
        [ 70, 269,  -2],
        [190, 380,  -4],
        [ 60, 274,  -3],
        [160, 335,  -3],
        [120, 281,  -2],
        [ 40, 332,  -5],
        [130, 350,  -3],
        [ 70, 319,  -4],
        [170, 342,  -3],
        [ 60, 275,  -2],
        [140, 356,  -3],
        [120, 363,  -4],
        [110, 336,  -3],
        [ 70, 262,  -3],
        [ 70, 277,  -2],
        [140, 323,  -2],
        [110, 304,  -3],
        [120, 356,  -3],
        [140, 305,  -3],
        [120, 301,  -2],
        [ 90, 327,  -3],
        [120, 308,  -3],
        [ 90, 328,  -3],
        [ 90, 290,  -3],
        [110, 285,  -3],
        [120, 351,  -4],
        [ 30, 273,  -3],
        [100, 291,  -4],
        [100, 340,  -4],
        [110, 322,  -3],
        [120, 292,  -4],
        [100, 349,  -4],


In [35]:
a[1]

[tensor([ 30, 277,  -2]),
 tensor([ 60, 280,  -3]),
 tensor([130, 345,  -4]),
 tensor([140, 359,  -4]),
 tensor([170, 340,  -2]),
 tensor([140, 319,  -2]),
 tensor([160, 357,  -4]),
 tensor([180, 334,  -2]),
 tensor([ 70, 269,  -2]),
 tensor([190, 380,  -4]),
 tensor([ 60, 274,  -3]),
 tensor([160, 335,  -3]),
 tensor([120, 281,  -2]),
 tensor([ 40, 332,  -5]),
 tensor([130, 350,  -3]),
 tensor([ 70, 319,  -4]),
 tensor([170, 342,  -3]),
 tensor([ 60, 275,  -2]),
 tensor([140, 356,  -3]),
 tensor([120, 363,  -4]),
 tensor([110, 336,  -3]),
 tensor([ 70, 262,  -3]),
 tensor([ 70, 277,  -2]),
 tensor([140, 323,  -2]),
 tensor([110, 304,  -3]),
 tensor([120, 356,  -3]),
 tensor([140, 305,  -3]),
 tensor([120, 301,  -2]),
 tensor([ 90, 327,  -3]),
 tensor([120, 308,  -3]),
 tensor([ 90, 328,  -3]),
 tensor([ 90, 290,  -3]),
 tensor([110, 285,  -3]),
 tensor([120, 351,  -4]),
 tensor([ 30, 273,  -3]),
 tensor([100, 291,  -4]),
 tensor([100, 340,  -4]),
 tensor([110, 322,  -3]),
 tensor([120

In [47]:
a = torch.randn((512,512))

In [48]:
a = a.repeat(3,1,1)

In [49]:
a.shape

torch.Size([3, 512, 512])

In [59]:
torch.stack([dataset[0][1], dataset[1][1]]).shape

torch.Size([2, 3])

In [60]:
endp = torch.tensor([30, 277,  -2])

In [63]:
endp = endp.unsqueeze(0)

In [66]:
endp.repeat(100, 1).shape

torch.Size([100, 3])

In [56]:
endp.repeat(100,100,100).shape

torch.Size([100, 100, 300])

In [57]:
endp.shape

torch.Size([3])